In [1]:
### PLEASE COPY NOTEBOOKS TO YOUR FOLDERS TO PREVENT COMMIT CONFLICTS
from dcicutils import ff_utils
from functions.notebook_functions import *
import json

# get key from keypairs.json
my_key = get_key('koray_data')
schema_name = get_schema_names(my_key) 
print('WORKING ON', my_key['server'], '\n')

##### COLLECT ITEMS TO Release #####
# use either a starting item to fetch all linked items

# Use a starting item to find linked ones
# starting_items = ['46db06ad-b399-4cf4-9acc-07b3e25ef132']
#add_items = get_query_or_linked(my_key, linked=starting_items)

# or a search query
#my_query = '/search/?q=GOLD&type=Item&limit=all'
#add_items = get_query_or_linked(my_key, query=my_query)

# if you want you can dump them to separate json files (will work as test insert)
# dump_to_json(add_items, destination folder)

my_query = '/search/?biosample.biosource.individual.organism.name=mouse&biosample.biosource_summary=ES-E14&experiment_type=in%20situ%20Hi-C&type=ExperimentHiC'
store = get_query_or_linked(my_key, query=my_query)
print(store.keys())
print(len([i['uuid'] for key in store for i in store[key]]))
print()

find_linked = ['a137500c-b54e-4076-9a6b-3628281d701e']
store = get_query_or_linked(my_key, linked=find_linked)
print(store.keys())
print(len([i['uuid'] for key in store for i in store[key]]))


WORKING ON https://data.4dnucleome.org 

dict_keys(['experiment_hi_c'])
3

dict_keys(['file_processed', 'award', 'file_format', 'user', 'static_section', 'protocol', 'ontology_term', 'biosample_cell_culture', 'biosource', 'organism', 'individual_mouse', 'biosample', 'lab', 'modification', 'ontology', 'imaging_path', 'microscope_setting_a2', 'vendor', 'experiment_mic', 'experiment_set_replicate', 'target'])
60


In [2]:
print(store['biosample'][0])

{'accession': '4DNBSSOXLHCL', 'uuid': 'f170ccae-b91a-43b2-81a1-4458a130874b', 'biosample_type': 'stem cells', 'external_references': [], 'description': '', '@id': '/biosamples/4DNBSSOXLHCL/', 'treatments': [], 'project_release': '2018-07-27', 'lab': '/labs/xavier-darzacq-lab/', 'link_id': '~biosamples~4DNBSSOXLHCL~', 'aliases': ['xavier-darzacq-lab:spot_on_biosample_jm8n4T2_halo_flag_sox2_4'], 'public_release': '2018-07-27', 'last_modified': {'date_modified': '2018-07-27T19:44:07.384216+00:00', 'modified_by': '/users/986b362f-4eb6-4a9c-8173-3ab267111888/'}, 'treatments_summary': 'None', 'schema_version': '1', 'date_created': '2018-07-20T20:29:51.569863+00:00', 'biosample_protocols': ['/protocols/5869253d-48bb-413c-8486-518d01e53400/'], 'award': '/awards/1U01EB021236-01/', 'principals_allowed': {'view': ['system.Everyone'], 'audit': ['system.Everyone'], 'edit': ['group.admin']}, 'display_title': '4DNBSSOXLHCL', 'modifications_summary': 'Crispr', '@type': ['Biosample', 'Item'], 'biosourc

In [ ]:
# Sample code to Release the items, modify as you wish
action = input('Do you want to perform patches, or only reporting (True/False):')
change_status = 'released'

counter = 0
for a_type in store:
    for raw_data in store[a_type]:
        item_status = raw_data['status']
        if item_status in ['deleted']: 
            print('Deleted data', a_type ,raw_data['uuid'])
            continue
        if item_status in ['current', 'released', change_status]:
            print('Already has status', item_status, a_type ,raw_data['uuid'])
            continue
             
        counter +=1
        print(a_type, raw_data['status'], raw_data['uuid'])
        patch_data = {'status': change_status}
        if action:
            resp = 'n'
            if a_type in ['publication', 'user', 'award', 'vendor', 'organism', ]:
                q = 'Do you want to update item {} to status to CURRENT? (y/n):'.format(a_type+'/'+raw_data['uuid'])
                resp = input(q)
                if resp == 'y':
                    ff_utils.patch_metadata({'status': 'current'}, obj_id=raw_data['uuid'] ,key=my_auth)
            else:
                ff_utils.patch_metadata(patch_data, obj_id=raw_data['uuid'] ,key=my_auth)

            
print(counter)